In [3]:
import os
print(os.getcwd())

# Specify the path you want to change to
new_path = '/home/carlos-alfonzo/PycharmProjects/streamlitapp'

# Change the current working directory
os.chdir(new_path)

# Verify the change
print("Current Working Directory: ", os.getcwd())


/home/carlos-alfonzo/PycharmProjects/streamlitapp/pages
Current Working Directory:  /home/carlos-alfonzo/PycharmProjects/streamlitapp


In [59]:
import pdfplumber
import re
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

In [75]:
def file_path_fcn(file_path=None):
    if file_path is None:
        file_path = 'REPORTES/RICARDO.pdf'
    return file_path

file_path ='REPORTES/KENA.pdf'

file_path = file_path_fcn()
file_path

'REPORTES/RICARDO.pdf'

In [76]:
def pdf_a_texto(file_path):
    '''Conversion del pdf a texto'''
    with pdfplumber.open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

texto = pdf_a_texto(file_path)

In [77]:
def format_spanish_date(date_str):
    # Define Spanish month and day names
    days = ["Lunes", "Martes", "Miércoles", "Jueves",
            "Viernes", "Sábado", "Domingo"]
    months = ["enero", "febrero", "marzo", "abril", "mayo", "junio", "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]

    # Parse the input date
    date_obj = datetime.strptime(date_str, "%d/%m/%Y")

    # Get day of week (0=Monday, 6=Sunday)
    day_of_week = date_obj.weekday()

    # Format the date in Spanish
    return f"{days[day_of_week]}, {date_obj.day} de {months[date_obj.month - 1]} de {date_obj.year}"

# Example usage:
date_str = "13/10/1964"
formatted_date = format_spanish_date(date_str)
print(formatted_date)  # Output: Martes, 13 de octubre de 1964

Martes, 13 de octubre de 1964


In [78]:
# Converting the general dates to datetime format
def convertir_a_fecha(fecha_str):
    # All the dates from the PDF are in the format "DD/MM/YYYY"
    fecha = datetime.strptime(fecha_str, '%d/%m/%Y').date()
    return fecha

# Example usage
esempio = "13/10/1964"
convertir_a_fecha(esempio)


datetime.date(1964, 10, 13)

In [79]:
class DatosGenerales:
    def __init__(self, texto):
        self.texto = texto

    def nombrefcn(self):
    # Extract the substring between the found positions, adjust indices for correct slicing
        InicioStr = re.search(r"20\d{2}\s{1}", self.texto)
        FinalStr = "DD MM YYYY"
        # Find the starting and ending positions of the target substrings
        Inicio = InicioStr.span()[0]
        Final = texto.find(FinalStr)
        # Extract the substring between the found positions, adjust indices for correct slicing
        if Inicio != -1 and Final != -1:
            # + 5 since InicioStr length = length(20) + 2 + 1 = 5
            Nombre = self.texto[Inicio + 5: Final].strip()
        else:
            Nombre = ""
        print(f"NOMBRE: {Nombre}")
        self.Nombre = Nombre

    def nssfcn(self):
        InicioStr = "NSS: "
        FinalStr = "CURP:"

        Inicio = self.texto.find(InicioStr)
        Final = self.texto.find(FinalStr)

        if Inicio != -1 and Final != -1:
            nss = self.texto[Inicio + 5: Final - 28].strip()
        else:
            nss = ""
        print(f"NSS: {nss}")
        self.nss = nss

    def curpfcn(self):
        InicioStr = "CURP:"

        Inicio = self.texto.find(InicioStr)

        if Inicio != -1:
            curp = self.texto[Inicio +len(InicioStr): Inicio + 25].strip()
        else:
            curp = ''
        print(f"CURP: {curp}")
        self.curp = curp

    def fecha_nacimiento_fcn(self):
        YEAR_BIRTH = "19" + self.curp[4:6]
        MONTH_BIRTH = self.curp[6:8]
        DAY_BIRTH = self.curp[8:10]
        FECHA_NACIMIENTO = DAY_BIRTH + '/' + MONTH_BIRTH + '/' + YEAR_BIRTH
        # Convert string to datetime object
        FECHA_NACIMIENTO_date = convertir_a_fecha(FECHA_NACIMIENTO)
        # Convert string to spanish date string
        FECHA_NACIMIENTO = format_spanish_date(FECHA_NACIMIENTO)
        print(f"FECHA DE NACIMIENTO: {FECHA_NACIMIENTO}")
        self.fecha_nacimiento = FECHA_NACIMIENTO
        self.fecha_nacimiento_date = FECHA_NACIMIENTO_date

    def ano_inscripcionIMSS_fcn(self):
        TerminacionYear = self.nss[2:4]
        if int(TerminacionYear) < 50:
            AnoInicio = int('20'+ TerminacionYear)
        else:
            AnoInicio = int('19'+ TerminacionYear)
        print(f"AÑO DE INSCRIPCIÓN IMSS: {AnoInicio}")
        self.AnoInicio = AnoInicio

    def regimen_pertenecefcn(self):
        if self.AnoInicio > 1997:
            Regimen = 'Régimen 97'
        elif self.AnoInicio == 1997:
            Regimen = "Determinar el mes que inició, si es a partir de julio, entra en régimen 97"
        else:
            Regimen = 'Régimen 73'

        print(f"REGIMEN AL QUE PERTENECE: {Regimen}")
        self.Regimen = Regimen

    def semanas_cotizadas_descontadas_fcn(self):
        Iniciostr = re.search(r"\d{1,4}\s{1}\d{1,4}\s{1}\d{1,4}", texto)
        myrange = Iniciostr.span()
        risultati = self.texto[myrange[0]:myrange[1]]
        semanas_cotizadas = int(risultati.split()[0])
        semanas_descontadas = int(risultati.split()[1])
        semanas_reintegradas = int(risultati.split()[-1])
        semanas_totales = semanas_cotizadas - semanas_descontadas + semanas_reintegradas

        print(f"SEMANAS COTIZADAS: {semanas_cotizadas}")
        print(f"SEMANAS DESCONTADAS: {semanas_descontadas}")
        print(f"SEMANAS REINTEGRADAS: {semanas_reintegradas}")
        print(f"SEMANAS TOTALES: {semanas_totales}")

        self.semanas_cotizadas = semanas_cotizadas
        self.semanas_descontadas = semanas_descontadas
        self.semanas_reintegradas = semanas_reintegradas
        self.semanas_totales = semanas_totales

    def fecha_emision_reporte_fcn(self):
        texto_array = self.texto.split('\n')

        # Find the position of the string that ends with 'reporte' and select the next element that contains the date
        position_date_report = [i + 1 for i, s in enumerate(texto_array) if s.endswith('reporte')]
        out = texto_array[position_date_report[0]]
        out = out.replace(' ', '')
        self.fecha_emision_reporte_date = convertir_a_fecha(out)
        print(f"FECHA DE EMISIÓN DEL REPORTE: {out}, type: {type(out)}")

        self.fecha_emision_reporte = format_spanish_date(out)
        print(f"FECHA DE EMISIÓN DEL REPORTE: {self.fecha_emision_reporte}, type: {type(self.fecha_emision_reporte)}")

    def tabla_datos(self):
        self.nombrefcn()
        self.nssfcn()
        self.curpfcn()
        self.fecha_nacimiento_fcn()
        self.ano_inscripcionIMSS_fcn()
        self.regimen_pertenecefcn()
        self.semanas_cotizadas_descontadas_fcn()
        self.fecha_emision_reporte_fcn()

        out = pd.DataFrame({
            'NOMBRE': [self.Nombre],
            'NSS': [self.nss],
            'CURP': [self.curp],
            'FECHA DE NACIMIENTO': [self.fecha_nacimiento],
            'AÑO DE INSCRIPCIÓN IMSS': [self.AnoInicio],
            'REGIMEN AL QUE PERTENECE': [self.Regimen],
            'SEMANAS COTIZADAS': [self.semanas_cotizadas],
            'SEMANAS DESCONTADAS': [self.semanas_descontadas],
            'SEMANAS REINTEGRADAS': [self.semanas_reintegradas],
            'SEMANAS TOTALES': [self.semanas_totales],
            'FECHA DE EMISIÓN DEL REPORTE': [self.fecha_emision_reporte]
        }
        )
        out = out.T
        out.columns = ['DATOS PERSONALES']
        return out

# Example usage
Usuario = DatosGenerales(texto)
df = Usuario.tabla_datos()
df

NOMBRE: PADILLA CASTILLO RICARDO FRANCISCO
NSS: 20906500192
CURP: PACR651005HTSDSC07
FECHA DE NACIMIENTO: Martes, 5 de octubre de 1965
AÑO DE INSCRIPCIÓN IMSS: 1990
REGIMEN AL QUE PERTENECE: Régimen 73
SEMANAS COTIZADAS: 842
SEMANAS DESCONTADAS: 77
SEMANAS REINTEGRADAS: 0
SEMANAS TOTALES: 765
FECHA DE EMISIÓN DEL REPORTE: 02/08/2025, type: <class 'str'>
FECHA DE EMISIÓN DEL REPORTE: Sábado, 2 de agosto de 2025, type: <class 'str'>


,DATOS PERSONALES
NOMBRE,PADILLA CASTILLO RICARDO FRANCISCO
NSS,20906500192
CURP,PACR651005HTSDSC07
FECHA DE NACIMIENTO,"Martes, 5 de octubre de 1965"
AÑO DE INSCRIPCIÓN IMSS,1990
REGIMEN AL QUE PERTENECE,Régimen 73
SEMANAS COTIZADAS,842
SEMANAS DESCONTADAS,77
SEMANAS REINTEGRADAS,0
SEMANAS TOTALES,765


In [130]:
class FechasGenerales:
    def __init__(self, texto, FechaEmisionReporte):
        self.texto = texto
        self.FechaEmisionReporte = FechaEmisionReporte

    def tabla_fechas_generales(self):
        texto = self.texto
        FechaEmisionReporte = self.FechaEmisionReporte
        # Define the search string
        bajas_string = 'Fecha de baja'
        value_added = 12 # Assuming 12 characters for the date
        # Find the indices of the search string
        bajas = [m.start() for m in re.finditer(bajas_string, texto)]
        # Get the index of the first occurrence
        ultima_baja = bajas[0]
        # Calculate the end index
        ultima_final = ultima_baja + len(bajas_string) + value_added
        # Extract the desired substring
        sigue_cotizando = "vigente" in texto.lower()  #Check if sigue_cotizando is contained in the text
        # Initialize an array to store dates
        fecha_bajas = np.zeros(len(bajas), dtype=object)
        fecha_bajas_date = np.zeros(len(bajas), dtype=object)

        for idx, baja in enumerate(bajas):
            if idx == 0:
                if not sigue_cotizando:
                    fechas_ultima_baja = convertir_a_fecha(texto[ultima_baja + len(bajas_string):ultima_final].strip())
                    print(fechas_ultima_baja, type(fechas_ultima_baja))
                    fecha_bajas[idx] = fechas_ultima_baja
                    fecha_bajas_date[idx] = format_spanish_date(texto[ultima_baja + len(bajas_string):ultima_final].strip())
                else:
                    fecha_bajas[idx] = FechaEmisionReporte
                    fecha_bajas_date[idx] = format_spanish_date(fecha_bajas[idx])
                    fechas_ultima_baja = fecha_bajas[idx]

                if not isinstance(fechas_ultima_baja, date):
                    fechas_ultima_baja = fechas_ultima_baja.date()
            else:
                start_idx = baja + len(bajas_string)
                end_idx = start_idx + value_added  # Assuming 12 characters for the date
                fecha_bajas[idx] = texto[start_idx:end_idx].strip()
                fecha_bajas_date[idx] = format_spanish_date(fecha_bajas[idx])
                fecha_bajas[idx] = convertir_a_fecha(fecha_bajas[idx])

        altas_string = 'Fecha de alta'
        altas = [m.start() for m in re.finditer(altas_string, texto)]

        ultima_alta = altas[0]
        ultima_alta_final = ultima_alta + len(altas_string) + value_added

        fechas_ultima_alta = texto[ultima_alta + len(altas_string):ultima_alta_final].strip()

        fecha_altas = np.zeros(len(altas), dtype=object)
        fecha_altas_date = np.zeros(len(altas), dtype=object)
        dias_transcurridos_cotizados = np.zeros(len(altas), dtype=object)
        semanas_transcurridas_cotizadas = np.zeros(len(altas), dtype=object)

        for idx, alta in enumerate(altas):
            start_idx = alta + len(altas_string)
            end_idx = start_idx + value_added
            fecha_altas[idx] = convertir_a_fecha(texto[start_idx:end_idx].strip())
            fecha_altas_date[idx] = format_spanish_date(texto[start_idx:end_idx].strip())


            dias_transcurridos_cotizados[idx] = (fecha_bajas[idx] - fecha_altas[idx]).days
            semanas_transcurridas_cotizadas[idx] = dias_transcurridos_cotizados[idx] // 7

            inicio_patron_str = 'Nombre del patrón'
            final_patron_str = 'Registro Patronal'
            final_patron_str_entidad = 'Entidad federativa'
            inicio_patron = [m.start() for m in re.finditer(inicio_patron_str, texto)]
            final_patron = [m.start() for m in re.finditer(final_patron_str, texto)]

            if len(final_patron) != len(inicio_patron):
                missing_registro_patronal = [m.start() for m in re.finditer(final_patron_str_entidad, texto)]
                final_patron.append(missing_registro_patronal[-1])

            patrones = []
            # using list comprehension
            patrones = [texto[inicio + len(inicio_patron_str):final_patron[idx] - 1].strip()
                        for idx, inicio in enumerate(inicio_patron)]

            inicio_ef_str = 'Entidad federativa'
            final_ef_str = 'Fecha de alta'
            inicio_ef = [m.start() for m in re.finditer(inicio_ef_str, texto)]
            final_ef = [m.start() for m in re.finditer(final_ef_str, texto)]

            entidad_federativa = []

            for idx, inicio in enumerate(inicio_ef):
                start_idx = inicio + len(inicio_ef_str)
                end_idx = final_ef[idx] - 1
                entidad_federativa.append(texto[start_idx:end_idx].strip())

            # Alternatively, using list comprehension
            entidad_federativa = [texto[inicio + len(inicio_ef_str):final_ef[idx] - 1].strip()
                                for idx, inicio in enumerate(inicio_ef)]

            FechasGenerales_num = pd.DataFrame({
            'Fecha de alta': fecha_altas,
            'Fecha de baja': fecha_bajas,
            'Dias transcurridos cotizados': dias_transcurridos_cotizados,
            'Semanas transcurridas cotizadas': semanas_transcurridas_cotizadas,
            'Patrones': patrones,
            'Entidad federativa': entidad_federativa
        })
            FechasGenerales_date = pd.DataFrame({
            'Fecha de alta': fecha_altas_date,
            'Fecha de baja': fecha_bajas_date,
            'Dias transcurridos cotizados': dias_transcurridos_cotizados,
            'Semanas transcurridas cotizadas': semanas_transcurridas_cotizadas,
            'Patrones': patrones,
            'Entidad federativa': entidad_federativa
        })
        return FechasGenerales_num, FechasGenerales_date, sigue_cotizando, fechas_ultima_baja




fechas_generales = FechasGenerales(texto=texto, FechaEmisionReporte=Usuario.fecha_emision_reporte)
fechasGeneralesNumerico, fechasGeneralesDate, sigueCotizando, fechasUltimaBaja = fechas_generales.tabla_fechas_generales()

display(fechasGenerales)
display(fechasGeneralesDate)
print(sigueCotizando)
print(fechasUltimaBaja)


2025-03-22 <class 'datetime.date'>


,Fecha de alta,Fecha de baja,Dias transcurridos cotizados,Semanas transcurridas cotizadas,Patrones,Entidad federativa
0,2024-09-18,2025-03-22,185,26,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
1,2009-12-01,2021-05-14,4182,597,INCLUYENDO MEXICO,CIUDAD DE MÉXICO
2,2008-05-12,2009-10-01,507,72,"MACSERNOM, S. DE R.L. DE C.V.",NUEVO LEÓN
3,1994-08-01,1995-03-02,213,30,VALORES MEXICANOS CASA DE BOLSA,CIUDAD DE MÉXICO
4,1992-02-17,1993-08-01,531,75,IXE CASA DE BOLSA S.A DE C.V. GRUPO FINANCIERO...,CIUDAD DE MÉXICO
5,1991-09-02,1993-02-01,518,74,"EL FINANCIERO, S.A. DE C.V.",CIUDAD DE MÉXICO
6,1990-02-01,1990-05-16,104,14,"INDUSTRIAS UNIDAS, S.A. DE C.V.",CIUDAD DE MÉXICO


,Fecha de alta,Fecha de baja,Dias transcurridos cotizados,Semanas transcurridas cotizadas,Patrones,Entidad federativa
0,"Miércoles, 18 de septiembre de 2024","Sábado, 22 de marzo de 2025",185,26,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
1,"Martes, 1 de diciembre de 2009","Viernes, 14 de mayo de 2021",4182,597,INCLUYENDO MEXICO,CIUDAD DE MÉXICO
2,"Lunes, 12 de mayo de 2008","Jueves, 1 de octubre de 2009",507,72,"MACSERNOM, S. DE R.L. DE C.V.",NUEVO LEÓN
3,"Lunes, 1 de agosto de 1994","Jueves, 2 de marzo de 1995",213,30,VALORES MEXICANOS CASA DE BOLSA,CIUDAD DE MÉXICO
4,"Lunes, 17 de febrero de 1992","Domingo, 1 de agosto de 1993",531,75,IXE CASA DE BOLSA S.A DE C.V. GRUPO FINANCIERO...,CIUDAD DE MÉXICO
5,"Lunes, 2 de septiembre de 1991","Lunes, 1 de febrero de 1993",518,74,"EL FINANCIERO, S.A. DE C.V.",CIUDAD DE MÉXICO
6,"Jueves, 1 de febrero de 1990","Miércoles, 16 de mayo de 1990",104,14,"INDUSTRIAS UNIDAS, S.A. DE C.V.",CIUDAD DE MÉXICO


False
2025-03-22


# Historial Laboral Desglosado

In [131]:
def HistorialLaboralTabla(texto):
    # Define the search string
    BloqueInicio = 'Nombre del patrón'
    BloqueFinal = '* Valor del último salario base de cotización diario en pesos.'

    # Find occurrences of the start and end blocks
    Inicio = [m.start() for m in re.finditer(re.escape(BloqueInicio), texto)]
    print(Inicio)
    Final = [m.start() for m in re.finditer(re.escape(BloqueFinal), texto)]
    # print(Final, BloqueFinal in texto)
    print(texto)
    HistoriaLaboral = []

    # Extract information between blocks
    for start, end in zip(Inicio, Final):
        substring = texto[start + len(BloqueInicio):end].strip()
        output = [line.strip() for line in substring.splitlines() if len(line.strip()) > 1]
        HistoriaLaboral.append(output)

    # Initialize an empty DataFrame for the final table
    HistoriaLaboralTable = pd.DataFrame()
    # target_strings = {'ALTA', 'REINGRESO', 'MODIFICACION DE SALARIO', 'BAJA'}
    target_strings = {'ALTA', 'REINGRESO', 'MODIFICACION', 'BAJA'}
    Movimiento = list()
    FechaMovimiento = list()
    Sueldo = list()
    Empleador = list()
    EntidadFederativa = list()
    for entry in HistoriaLaboral:
        for line in entry:
            # if any(string in line for string in target_strings):
            if any(string in line.split()[0] for string in target_strings):
                Empleador.append(entry[0])
                EntidadFederativa.append(entry[2].split()[2:])
                array = line.split()
                if array[0] == 'MODIFICACION':
                    Movimiento.append('MODIFICACION DE SALARIO')
                else:
                    Movimiento.append(array[0])
                FechaMovimiento.append(array[-3])
                Sueldo.append(array[-1])

    # Convert 'Sueldo' elements to float before formatting
    Sueldo_formatted = [float(s) for s in Sueldo]
    HistoriaLaboralTable_str = pd.DataFrame({
        'Movimiento': Movimiento,
        'Fecha de Movimiento': FechaMovimiento,
        'Sueldo': Sueldo, #np.vectorize(lambda x: "${:,.2f}".format(x))(Sueldo_formatted),
        'Empleador': Empleador,
        'Entidad Federativa': EntidadFederativa
    })

    HistoriaLaboralTable_str['Entidad Federativa'] = HistoriaLaboralTable_str['Entidad Federativa'].apply(lambda x: " ".join(x))

    HistoriaLaboralTable_num = HistoriaLaboralTable_str.copy()
    HistoriaLaboralTable_num['Sueldo'] = Sueldo_formatted
    return HistoriaLaboralTable_str, HistoriaLaboralTable_num

HistoriaLaboralTable_str, HistoriaLaboralTable_num = HistorialLaboralTabla(texto)
def HistorialLaboralDesglosada_fcn(texto, sigueCotizando, FechasUltimaBaja):
    # Define the search string
    HistoriaLaboralTable, _ = HistorialLaboralTabla(texto)

    from datetime import timedelta
    Bajas = np.where(HistoriaLaboralTable['Movimiento'] == 'BAJA')[0]
    FechaFinal = HistoriaLaboralTable['Fecha de Movimiento'].values

    for idx in range(len(HistoriaLaboralTable)):
        FechaFinal[idx] = datetime.strptime(FechaFinal[idx], '%d/%m/%Y').date()
        if idx not in Bajas:
            FechaFinal[idx] =  FechaFinal[idx] - timedelta(days=1)


    FechaFinal = np.roll(FechaFinal, 1)

    if sigueCotizando:
        FechaFinal[0] = FechasUltimaBaja

    HistoriaLaboralDesglosada = HistoriaLaboralTable.copy()
    HistoriaLaboralDesglosada['Fecha Final'] = FechaFinal
    HistoriaLaboralDesglosada.drop(Bajas, inplace=True)
    HistoriaLaboralDesglosada = HistoriaLaboralDesglosada.rename(columns={"Fecha de Movimiento": "Fecha Inicial"})

    HistoriaLaboralDesglosada["Fecha Inicial"] = pd.to_datetime(HistoriaLaboralDesglosada["Fecha Inicial"], format='%d/%m/%Y')

    HistoriaLaboralDesglosada["Fecha Inicial"] = HistoriaLaboralDesglosada["Fecha Inicial"] + timedelta(days = 1)
    # Move the column "FechaFinal" to be after "FechaInicial"
    cols = list(HistoriaLaboralDesglosada.columns)
    cols.insert(cols.index("Fecha Inicial") + 1, cols.pop(cols.index("Fecha Final")))
    HistoriaLaboralDesglosada = HistoriaLaboralDesglosada[cols]
    # We will change the date format for the column "Fecha Inicial"
    HistoriaLaboralDesglosada['Fecha Inicial'] = pd.to_datetime(HistoriaLaboralDesglosada['Fecha Inicial'], format='%d/%m/%Y').dt.date
    return HistoriaLaboralDesglosada

HistoriaLaboralTable = HistorialLaboralDesglosada_fcn(texto, sigueCotizando, fechasUltimaBaja)

print("Historial Laboral:")
HistoriaLaboralTable_str.head()

[481, 839, 1889, 2502, 3003, 3439, 4091]
Instituto Mexicano del Seguro Social
Constancia de Semanas Cotizadas en el IMSS e
historial de registros afiliatorios de la persona asegurada
Estimado(a), Fecha de emisión del reporte
02 / 08 / 2025
PADILLA CASTILLO RICARDO FRANCISCO
DD MM YYYY
NSS: 20906500192 Total de semanas cotizadas
CURP: PACR651005HTSDSC07 765
Tu detalle de semanas cotizadas
Semanas Descontadas
Semanas cotizadas IMSS Semanas Reintegradas (+)
(por disposición de recursos) (-)
842 77 0
Tu historia laboral
Nombre del patrón CARLOS DAVID ALFONZO UTRILLA
Registro Patronal A621154810
Entidad federativa CHIAPAS
Fecha de alta 18/09/2024 Fecha de baja 22/03/2025 Salario Base de Cotización * $ 334.0
Tipo de movimiento Fecha de movimiento Salario Base
BAJA 22/03/2025 $ 334.0
REINGRESO 18/09/2024 $ 334.0
* Valor del último salario base de cotización diario en pesos.
Nombre del patrón INCLUYENDO MEXICO
Registro Patronal Y544112110
Entidad federativa CIUDAD DE MÉXICO
Fecha de alta 01/12

,Movimiento,Fecha de Movimiento,Sueldo,Empleador,Entidad Federativa
0,BAJA,22/03/2025,334.0,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
1,REINGRESO,18/09/2024,334.0,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
2,BAJA,14/05/2021,2172.0,INCLUYENDO MEXICO,CIUDAD DE MÉXICO
3,MODIFICACION DE SALARIO,01/02/2020,2172.0,INCLUYENDO MEXICO,CIUDAD DE MÉXICO
4,MODIFICACION DE SALARIO,01/02/2019,2112.25,INCLUYENDO MEXICO,CIUDAD DE MÉXICO
